# NYC Taxi Fare Prediction

### **Table of Contents**
1. Download data
2. Read data using Pandas
3. Exploratory data anaysis and data cleaning
4. Feature engineering 
5. Understand temporal feature conversions 
6. Todo
6. Todo

## Import packages

In [3]:
import numpy as np
import seaborn as sb
import pandas as pd
import tensorflow as tf

## 1. Download data
Download data using ```wget```. A large volume of data is available at https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page, but we are going to use only ```yellow_tripdata_2020-06.csv```, which has 499,000 data points. 


In [1]:
# !wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-02.csv
# !wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-03.csv
# !wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-04.csv
# !wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-05.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-06.csv

--2020-11-16 22:49:25--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-06.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.106.29
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.106.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50277193 (48M) [text/csv]
Saving to: ‘yellow_tripdata_2020-06.csv.1’

pdata_2020-06.csv.1  35%[======>             ]  17.26M   728KB/s    eta 26s    ^C


## 2. Read data using Pandas

Let's try to peek the data using ```head``` method. The definition of each column names can be found from [here](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf). Also note that ```PULocationID``` and ```DOLocationID``` indicate pickup and dropoff zones. More details can be checked from [here](https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv).


In [5]:
data = pd.read_csv('./yellow_tripdata_2020-06.csv')
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-06-01 00:31:23,2020-06-01 00:49:58,1.0,3.6,1.0,N,140,68,1.0,15.5,3.0,0.5,4.00,0.0,0.3,23.30,2.5
1,1.0,2020-06-01 00:42:50,2020-06-01 01:04:33,1.0,5.6,1.0,N,79,226,1.0,19.5,3.0,0.5,2.00,0.0,0.3,25.30,2.5
2,1.0,2020-06-01 00:39:51,2020-06-01 00:49:09,1.0,2.3,1.0,N,238,116,2.0,10.0,0.5,0.5,0.00,0.0,0.3,11.30,0.0
3,1.0,2020-06-01 00:56:13,2020-06-01 01:11:38,1.0,5.3,1.0,N,141,116,2.0,17.5,3.0,0.5,0.00,0.0,0.3,21.30,2.5
4,1.0,2020-06-01 00:16:41,2020-06-01 00:29:30,1.0,4.4,1.0,N,186,75,1.0,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5


## 3. Exploratory data analysis

```describe``` is a very useful method to get summary statistics for the numeric fields of the data. Each column of the first row has different values because 


In [6]:
data.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,499043.000000,499043.000000,549760.000000,499043.000000,549760.000000,549760.000000,499043.000000,549760.000000,549760.000000,549760.000000,549760.000000,549760.000000,549760.000000,549760.000000,549760.000000
mean,1.598351,1.356148,4.104275,1.047214,157.636474,153.473989,1.373327,13.606734,1.023772,0.491298,1.762904,0.367066,0.296980,18.768912,1.967681
std,0.490232,1.016665,336.024280,1.095790,69.756787,73.842217,0.531616,13.521364,1.263818,0.080832,2.631495,1.751982,0.041437,15.178965,1.044792
min,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-216.000000,-4.500000,-0.500000,-36.300000,-28.750000,-0.300000,-216.300000,-2.500000
25%,1.000000,1.000000,1.010000,1.000000,107.000000,87.000000,1.000000,6.000000,0.000000,0.500000,0.000000,0.000000,0.300000,10.700000,2.500000
50%,2.000000,1.000000,1.860000,1.000000,151.000000,151.000000,1.000000,9.000000,0.500000,0.500000,1.500000,0.000000,0.300000,14.160000,2.500000
75%,2.000000,1.000000,3.660000,1.000000,234.000000,233.000000,2.000000,15.500000,2.500000,0.500000,2.750000,0.000000,0.300000,20.800000,2.500000
max,2.000000,9.000000,220386.230000,99.000000,265.000000,265.000000,5.000000,941.500000,87.560000,3.300000,422.680000,114.750000,0.300000,1141.100000,2.500000
